In [1]:
#from boson_data_lib import *
import numpy as np
import scipy.linalg as sl
import os
import pandas as pd
import glob

In [2]:
import h5py

In [3]:
data_dir = "../DATA/"
tests_dir = ""
new_data_dir = ""

In [6]:
def read_as_dataframe(filename, tests_dir):

    df = pd.DataFrame()
    
    print("Processing "+filename+"...")
    
    with h5py.File(tests_dir+filename, "r") as f:
        
        gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33']    
        #gammas = f.keys()

        for gamma in gammas:
            
            print(gamma)
            
            #init_states = f[gamma].keys()
            init_states = ["D"+str(i) for i in range(1, 11)] 
            
            for state in init_states:
                
                Fidelity = f[gamma][state]["Fidelity"][()]
                #tr_dist = f[gamma][state]["tr_dist_grnd"][()]
                time = f[gamma][state]["time"][()]

                ser_len = len(Fidelity)

                gamma_column = [gamma] * ser_len
                state_column = [state] * ser_len

                #tr_dist_column = [tr_dist] * ser_len      
                
                gamma_time_column = np.array(time)*float(gamma) 

                d_ser = {'Gamma': gamma_column,
                         'State': state_column,
                         'Time': time, 
                         'gt': gamma_time_column,
                         'Fidelity': Fidelity,
                         'Infidelity': 1-Fidelity#,
                         #'Distance': tr_dist_column
                         }

                df_ser = pd.DataFrame(data = d_ser)   
                df = pd.concat([df, df_ser])
    
    print(" done!")
    
    return df

In [15]:

test_file3 = "POP_NEW_TSSOS_KOS_treshold_1e-3_FROB_QO_2024-Aug-08_at_14-37.h5"
test_file4 = "POP_NEW_TSSOS_KOS_treshold_1e-4_FROB_QO_2024-Aug-08_at_13-42.h5"
test_file5 = "POP_NEW_TSSOS_KOS_treshold_1e-5_FROB_QO_2024-Aug-08_at_15-16.h5"
test_file6 = "POP_NEW_TSSOS_KOS_treshold_1e-6_FROB_QO_2024-Aug-08_at_16-02.h5"

test_file8 = "POP_NEW_TSSOS_KOS_treshold_1e-8_FROB_QO_2024-Aug-09_at_04-04.h5"


In [16]:
# Your list of gamma values
#gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33'] 

df3 = read_as_dataframe(test_file3, tests_dir)
df4 = read_as_dataframe(test_file4, tests_dir)
df5 = read_as_dataframe(test_file5, tests_dir)
df6 = read_as_dataframe(test_file6, tests_dir)
df8 = read_as_dataframe(test_file8, tests_dir)

# List of your dataframes
dataframes = [df3, df4, df5, df6, df8]  # Replace with your actual dataframe variables
dataframe_names = ['1e-3', '1e-4', '1e-5', '1e-6']  # Names corresponding to each dataframe

# Your list of gamma values, converted to floats to match the dataframe
gammas = [0.079477, 0.25133, 0.79477, 2.5133, 7.9477, 25.133, 79.477, 251.33] 

# Create an empty list to store the best time series for each gamma
best_time_series = []

# Iterate over each gamma value
for gamma in gammas:
    best_mean_fidelity = -float('inf')
    best_series = None
    best_source = None
    
    # Check each dataframe for the best mean fidelity for the current gamma
    for df, name in zip(dataframes, dataframe_names):
        # Ensure that the Gamma column is of type float for correct comparison
        df['Gamma'] = df['Gamma'].astype(float)
        
        # Filter the dataframe for the current gamma
        gamma_df = df[df['Gamma'] == gamma]
        
        if not gamma_df.empty:
            # Calculate the mean fidelity for the current gamma in the current dataframe
            mean_fidelity = gamma_df['Fidelity'].mean()
            
            # Check if this is the best mean fidelity so far
            if mean_fidelity > best_mean_fidelity:
                best_mean_fidelity = mean_fidelity
                best_series = gamma_df
                best_source = name
    
    # Since we know a valid series exists, we simply add the best one
    best_series = best_series.copy()
    best_series['Source'] = best_source
    best_time_series.append(best_series)

# Combine all the best time series into one dataframe
final_df = pd.concat(best_time_series, ignore_index=True)


Processing POP_NEW_TSSOS_KOS_treshold_1e-3_FROB_QO_2024-Aug-08_at_14-37.h5...
0.079477
0.25133
0.79477
2.5133
7.9477
25.133
79.477
251.33
 done!
Processing POP_NEW_TSSOS_KOS_treshold_1e-4_FROB_QO_2024-Aug-08_at_13-42.h5...
0.079477
0.25133
0.79477
2.5133
7.9477
25.133
79.477
251.33
 done!
Processing POP_NEW_TSSOS_KOS_treshold_1e-5_FROB_QO_2024-Aug-08_at_15-16.h5...
0.079477
0.25133
0.79477
2.5133
7.9477
25.133
79.477
251.33
 done!
Processing POP_NEW_TSSOS_KOS_treshold_1e-6_FROB_QO_2024-Aug-08_at_16-02.h5...
0.079477
0.25133
0.79477
2.5133
7.9477
25.133
79.477
251.33
 done!
Processing POP_NEW_TSSOS_KOS_treshold_1e-8_FROB_QO_2024-Aug-09_at_04-04.h5...
0.079477
0.25133
0.79477
2.5133
7.9477
25.133
79.477
251.33
 done!


In [17]:
pkl_name = "dataframe_combined.pkl"

final_df.to_pickle(tests_dir+pkl_name)

In [18]:
pkl_name

'dataframe_combined.pkl'